# Export satellite images

***Note August 15th 2023**: due to constraints imposed by Google Earth Engine on data exports, this script has become excessively slow to run in its current form (estimated >300 h). The notebook below was what we ran to produce the results in our paper, but for the sake of usability we are currently developing a faster, equivalent setup for exporting and loading images. This script will be included alongside the original one as soon as it is ready.*

## Table of Contents
- [Pre-requisites](#pre-requisites)
- [Instructions](#instructions)
- [Imports and initialization](#imports-and-initialization)
- [Prepare the survey data](#prepare-the-survey-data)
- [Download the images](#download-the-images)

## Pre-requisites
Register an account on [Google Earth Engine (GEE)](https://earthengine.google.com/). You will need to provide a Gmail account. Once you have registered, you will need to [sign up for the Google Earth Engine API](https://signup.earthengine.google.com/#!/). This can take a few days to be approved. Once you have been approved, you will be able to use the GEE API.

## Instructions
This notebook exports the Landsat and Nightlight images used as input data for the various models in the project from Google Earth Engine (GEE) to Google Cloud Storage (GCS). The exported images take up about 230 GB of disk space. After the images have been exported to GCS they will have to be downloaded into the data directory as specified in [config.ini](../config.ini).

## Imports and initialization

Import the necessary libraries and config values.

In [1]:
import ee
import pandas as pd
import os

In [2]:
import configparser

In [3]:
import ee
import pandas as pd
import os
#from gee_utils import export_images, wait_on_tasks
import configparser

# Read config file
config = configparser.ConfigParser()
config.read('config.ini')

[]

In [6]:
#ee.Authenticate()

Before using the Earth Engine API, you must authenticate your credentials. Once you have run the following cell, you will be prompted to click on a link and copy a code into the text box. This will authenticate your credentials and allow you to use the Earth Engine API. You only need to do this once, unless prompted to do so again. Make sure that you log in using a Google account which has access to the GCS bucket defined in config.ini.

In [7]:
#ee.Initialize(project='ee-stevenndungu')

Initilaize the Earth Engine API with the high volume end-point. See [here](https://developers.google.com/earth-engine/cloud/highvolume) for more information.

In [8]:
#ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

## Prepare the survey data

Read the csv file with survey points

In [4]:
#data_dir = config['PATHS']['DATA_DIR']
dhs_cluster_file_path = r'G:\My Drive\temporal-eo-wealth\data\dhs_clusters.csv'#os.path.join(data_dir, 'dhs_clusters.csv')
#G:\My Drive\temporal-eo-wealth\data\dhs_clusters.csv
df = pd.read_csv(dhs_cluster_file_path)
df.head(20)

,country,survey_start_year,year,lat,lon,GID_1,GID_2,households,rural,iwi
0,angola,2006,2006,-12.169283,13.859255,AGO.2,AGO.2.9,20,0,68.096300
1,angola,2006,2006,-12.361700,14.778201,AGO.2,AGO.2.2,19,1,10.609053
2,angola,2006,2006,-10.827451,14.342114,AGO.7,AGO.7.1,25,1,29.973928
3,angola,2006,2006,-9.991311,14.898295,AGO.7,AGO.7.5,1,1,7.704135
4,angola,2006,2006,-11.518893,14.750437,AGO.7,AGO.7.10,24,1,13.371826
5,angola,2006,2006,-11.346350,15.073653,AGO.7,AGO.7.12,24,1,10.653362
6,angola,2006,2006,-12.167753,13.845458,AGO.2,AGO.2.9,24,0,62.009042
7,angola,2006,2006,-12.747312,13.607121,AGO.2,AGO.2.3,22,0,66.059021
8,angola,2006,2006,-11.502919,14.060841,AGO.7,AGO.7.11,25,0,51.727994
9,angola,2006,2006,-13.130185,13.109428,AGO.2,AGO.2.1,24,1,20.937783


In [5]:
len(df.country.unique())

36

In [6]:
df.country.unique()

array(['angola', 'benin', 'burkina_faso', 'burundi', 'cameroon',
       'central_african_republic', 'chad', 'comoros',
       'democratic_republic_of_congo', 'egypt', 'eswatini', 'ethiopia',
       'gabon', 'ghana', 'guinea', 'ivory_coast', 'kenya', 'lesotho',
       'liberia', 'madagascar', 'malawi', 'mali', 'morocco', 'mozambique',
       'namibia', 'niger', 'nigeria', 'rwanda', 'senegal', 'sierra_leone',
       'south_africa', 'tanzania', 'togo', 'uganda', 'zambia', 'zimbabwe'],
      dtype=object)

In [11]:
import numpy as np
np.sort(df.year.unique())
# Filter the DataFrame to include only the years 2000 to 2020

array([1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019], dtype=int64)

Get a list of all the country-year combinations included in the dataset

In [12]:
surveys = list(df.groupby(['country', 'year']).groups.keys())

In [13]:
surveys[0:5]

[('angola', 2006),
 ('angola', 2007),
 ('angola', 2010),
 ('angola', 2011),
 ('angola', 2015)]

In [14]:
country = 'angola'
year = 2015
test_df = df[(df['country'] == country) & (df['year'] == year)].sample(10, random_state=0)
test_df

,country,survey_start_year,year,lat,lon,GID_1,GID_2,households,rural,iwi
479,angola,2015,2015,-13.268225,16.286991,AGO.9,AGO.9.9,22,1,7.122878
632,angola,2015,2015,-7.926358,15.041177,AGO.17,AGO.17.13,8,0,37.609296
400,angola,2015,2015,-5.634583,12.197471,AGO.4,AGO.4.3,23,0,50.556994
599,angola,2015,2015,-10.671992,22.191444,AGO.15,AGO.15.5,9,1,5.111648
621,angola,2015,2015,-15.102504,12.148766,AGO.16,AGO.16.3,22,0,31.604586
426,angola,2015,2015,-9.291030,14.910762,AGO.6,AGO.6.6,17,0,33.691914
461,angola,2015,2015,-17.254237,15.359872,AGO.8,AGO.8.2,22,1,14.780055
469,angola,2015,2015,-12.748934,15.683734,AGO.9,AGO.9.5,13,0,50.489750
656,angola,2015,2015,-6.263827,14.245849,AGO.18,AGO.18.2,26,0,72.498381
401,angola,2015,2015,-5.599102,12.171812,AGO.4,AGO.4.3,24,0,70.907635


To make sure that you have all the permissions, libraries, etc. before starting the big list of tasks, run this test case which exports the first 10 clusters for a given survey. It shouldn't take more than 10 minutes.

In [ ]:
def test_export(df, country, year):
    test_df = df[(df['country'] == country) & (df['year'] == year)].sample(10, random_state=0)
    test_tasks = export_images(test_df,
                               country=country,
                               year=year,
                               export_folder=config['GCS']['EXPORT_FOLDER'],  # 'data/dhs_tfrecords_raw',
                               export='gcs',
                               bucket=config['GCS']['BUCKET'],
                               ms_bands=['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP1'],
                               include_nl=True,
                               start_year=1990,
                               end_year=2020,
                               span_length=3,
                               chunk_size=5)
    wait_on_tasks(test_tasks, poll_interval=60)

test_export(df, surveys[0][0], surveys[0][1])

Note that even if the "wait_on_tasks" method fails the tasks have still been started and are running in the GEE. If the task completes succesfully (seen either through wait_on_tasks or in the [GEE editor](https://code.earthengine.google.com/ )) you can continue to the next step.

## Export the images

This section sets up and queues all the tasks for your survey data. It will take a little while to load, but once it's executed you're done. The exports are now carried out in GEE and you can monitor the exports in the [GEE editor](https://code.earthengine.google.com/). Note that some of these tasks will most likely fail with memory errors. When one of them fail the next couple of ones (maybe next five) are also fails more likely to do so. It happens seamingly at random and there's nothing we've been able to do about it. After everyone are done I will rerun the exports only on the survey points which are missing in the GCS bucket, so unless a lot of them fail (for Egypt 2014 it was about 10%) this is nothing you need to worry about for now.

### Start missing downloads

Sometimes the connnection aborts before all tasks have been started. I've updated the script to first check which tasks have already been initiated before starting the rest.

In [48]:
latest_tasks = {}
for survey in surveys:
    latest_tasks[survey] = -1

Get the latest task already downloaded into the GCS bucket for each survey:

In [49]:
for survey in surveys:
    files_path = f'gs://{config['GCS']['BUCKET']}/{config['GCS']['EXPORT_FOLDER']}/{survey[0]}_{survey[1]}'
    files_in_bucket = !gsutil ls {files_path}*
    if files_in_bucket[-1].startswith(files_path):
        latest_file = files_in_bucket[-1]
        latest_file_nr = int(latest_file[len(files_path)+1:len(files_path)+5])
        latest_tasks[survey] = latest_file_nr

In [50]:
print('Latest tasks already in bucket:\n', latest_tasks)

Latest tasks already in bucket:
 {('madagascar', 2020): 1304, ('ethiopia', 2020): 1005}


Get the latest task started in GEE for each survey:

In [51]:
# Get task list from GEE
gee_tasks = !earthengine task list

# Loop over these tasks. Save the latest in "last_tasks", if it's higher than what is already in the GCS bucket.
for line in gee_tasks:
    if 'Export.table' in line:
        task = line.split()[2]
        survey_string = task.split('_')[:2]
        survey = (survey_string[0], int(survey_string[1]))
        if survey not in surveys:
            continue
        task_nr = int(task.split('_')[2][:4])
        if task_nr > latest_tasks[survey]:
            latest_tasks[survey] = task_nr

In [52]:
print('Latest tasks already started in GEE:\n', latest_tasks)

Latest tasks already started in GEE:
 {('madagascar', 2020): 1304, ('ethiopia', 2020): 1005}


Start the remaining tasks. If the connection is aborted before all tasks have started, please rerun this section of the notebook (from the "Export the images").

In [ ]:
chunk_size = 5
all_tasks = dict()

for survey in surveys:
    last_started = latest_tasks[survey]
    survey_df = df[(df['country'] == survey[0]) & (df['year'] == survey[1])]
    expected_nr_of_tasks = int(math.ceil(len(survey_df) / chunk_size))
    if last_started < expected_nr_of_tasks - 1:
        # Some tasks have not been started. Starts them here:
        country = survey[0]
        year = survey[1]
        already_in_bucket = list(range(last_started + 1))
        survey_tasks = export_images(df,
                                     country=country,
                                     year=year,
                                     export_folder=config['GCS']['EXPORT_FOLDER'],
                                     export='gcs',
                                     bucket=config['GCS']['BUCKET'],
                                     ms_bands=['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP1'],
                                     include_nl=True,
                                     start_year=1990,
                                     end_year=2020,
                                     span_length=3,
                                     chunk_size=5,
                                     already_in_bucket=already_in_bucket)
        all_tasks.update(survey_tasks)

If you prefer, you also monitor the tasks in the notebook with the "wait_on_tasks" method.

In [ ]:
wait_on_tasks(all_tasks, poll_interval=60)